In [1]:
import json
import geojson 
import geopandas
import pandas as pd
import numpy as np
import os
import datetime as dt

from lmd.lib import Collection, Shape
from lmd import tools
from PIL import Image
from lmd.lib import SegmentationLoader

import tifffile
import string

from shapely.geometry import Point, LineString

In [2]:
# show current anaconda env
print ("Current Conda Environment: " + str(os.environ['CONDA_DEFAULT_ENV']))

Current Conda Environment: py-lmd-env


### from samples and wells to 384 well plate 

In [ ]:
#exterior two rows and columns will be kept free to add water to prevent evaporation

In [4]:
list_of_acceptable_wells =[]
for row in list(string.ascii_uppercase[2:14]):
    for column in range(2,22):
        list_of_acceptable_wells.append(str(row) + str(column))

print('checking the first five wells: ' + str(list_of_acceptable_wells[:5]))
print('checking the last five wells: ' + str(list_of_acceptable_wells[-5:]))

checking the first five wells: ['C2', 'C3', 'C4', 'C5', 'C6']
checking the last five wells: ['N17', 'N18', 'N19', 'N20', 'N21']


In [46]:
rows = string.ascii_uppercase[:16]
rows_list = [i for i in rows]
rows_list[:]

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P']

In [21]:
columns_1_24 = [str(i) for i in range(1,25)]

In [22]:
columns_1_24[:5]

['1', '2', '3', '4', '5']

In [25]:
#create dataframe with columns names 1->24 and row names A->P
df = pd.DataFrame(0,columns=columns_1_24, index=rows_list)

In [26]:
df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#samples and wells dictionary example

In [27]:
samples_and_wells = {
 '187_HBC_mid_3': 'C3',
 '187_HBC_nVEC_1': 'C4',
 '187_HBC_nVEC_2': 'C5',
 '187_HBC_mid_2': 'C6',
 '187_HBC_nTroph_3': 'C7',
 '187_HBC_nTroph_1': 'C8',
 '187_HBC_mid_1': 'C9',
 '187_HBC_nVEC_3': 'C10',
 '187_HBC_nTroph_2': 'C11',
 '108_STB_nVEC_1': 'F3',
 '108_STB_nVEC_2': 'F4',
 '108_STB_nVEC_3': 'F5',
 '108_STB_gen_1': 'F6',
 '108_STB_gen_2': 'F7',
 '108_STB_gen_3': 'F8',
 '108_CTB_1_1': 'F9',
 '108_CTB_2_1': 'F10',
 '108_CTB_3_1': 'F11',
 '108_HBC_nTroph_1': 'G3',
 '108_HBC_nTroph_2': 'G4',
 '108_HBC_nTroph_3': 'G5',
 '108_HBC_nVEC_1': 'G6',
 '108_HBC_nVEC_2': 'G7',
 '108_HBC_nVEC_3': 'G8',
 '108_HBC_mid_1': 'G9',
 '108_HBC_mid_2': 'G10',
 '108_HBC_mid_3': 'G11',
 '108_CCT_1': 'H3',
 '108_CCT_2': 'H4',
 '108_CCT_3': 'H5',
 '108_VEC_1': 'H6',
 '108_VEC_2': 'H7',
 '108_VEC_3': 'H8',
 '108_Str_1': 'H9',
 '108_Str_2': 'H10',
 '108_Str_3': 'H11',
 '187_STB_nVEC_1': 'D3',
 '187_STB_nVEC_2': 'D4',
 '187_STB_nVEC_3': 'D5',
 '187_STB_gen_1': 'D6',
 '187_STB_gen_2': 'D7',
 '187_STB_gen_3': 'D8',
 '187_CTB_1': 'D9',
 '187_CTB_2': 'D10',
 '187_CTB_3': 'D11',
 '187_CCT_1': 'E3',
 '187_CCT_2': 'E4',
 '187_CCT_3': 'E5',
 '187_VEC_1': 'E6',
 '187_VEC_2': 'E7',
 '187_VEC_3': 'E8',
 '187_str_1': 'E9',
 '187_str_2': 'E10',
 '187_str_3': 'E11'}

In [ ]:
Get value at specified row/column pair

df.at[4, 'B']

In [42]:
#I should be able to add the key of those entries following the value
#create dataframe with columns names 1->24 and row names A->P
df = pd.DataFrame(0,columns=columns_1_24, index=rows_list)

for i in samples_and_wells:
    location = samples_and_wells[i]
    #print (location)
    df.at[location[0],location[1:]] = i
    

In [43]:
df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C,0,0,187_HBC_mid_3,187_HBC_nVEC_1,187_HBC_nVEC_2,187_HBC_mid_2,187_HBC_nTroph_3,187_HBC_nTroph_1,187_HBC_mid_1,187_HBC_nVEC_3,...,0,0,0,0,0,0,0,0,0,0
D,0,0,187_STB_nVEC_1,187_STB_nVEC_2,187_STB_nVEC_3,187_STB_gen_1,187_STB_gen_2,187_STB_gen_3,187_CTB_1,187_CTB_2,...,0,0,0,0,0,0,0,0,0,0
E,0,0,187_CCT_1,187_CCT_2,187_CCT_3,187_VEC_1,187_VEC_2,187_VEC_3,187_str_1,187_str_2,...,0,0,0,0,0,0,0,0,0,0
F,0,0,108_STB_nVEC_1,108_STB_nVEC_2,108_STB_nVEC_3,108_STB_gen_1,108_STB_gen_2,108_STB_gen_3,108_CTB_1_1,108_CTB_2_1,...,0,0,0,0,0,0,0,0,0,0
G,0,0,108_HBC_nTroph_1,108_HBC_nTroph_2,108_HBC_nTroph_3,108_HBC_nVEC_1,108_HBC_nVEC_2,108_HBC_nVEC_3,108_HBC_mid_1,108_HBC_mid_2,...,0,0,0,0,0,0,0,0,0,0
H,0,0,108_CCT_1,108_CCT_2,108_CCT_3,108_VEC_1,108_VEC_2,108_VEC_3,108_Str_1,108_Str_2,...,0,0,0,0,0,0,0,0,0,0
I,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
samples_and_wells = {
 '187_HBC_mid_3': 'C3',
 '187_HBC_nVEC_1': 'C4',
 '187_HBC_nVEC_2': 'C5',
 '187_HBC_mid_2': 'C6',
 '187_HBC_nTroph_3': 'C7',
 '187_HBC_nTroph_1': 'C8',
 '187_HBC_mid_1': 'C9',
 '187_HBC_nVEC_3': 'C10',
 '187_HBC_nTroph_2': 'C11',
 '108_STB_nVEC_1': 'F3',
 '108_STB_nVEC_2': 'F4',
 '108_STB_nVEC_3': 'F5',
 '108_STB_gen_1': 'F6',
 '108_STB_gen_2': 'F7',
 '108_STB_gen_3': 'F8',
 '108_CTB_1_1': 'F9',
 '108_CTB_2_1': 'F10',
 '108_CTB_3_1': 'F11',
 '108_HBC_nTroph_1': 'G3',
 '108_HBC_nTroph_2': 'G4',
 '108_HBC_nTroph_3': 'G5',
 '108_HBC_nVEC_1': 'G6',
 '108_HBC_nVEC_2': 'G7',
 '108_HBC_nVEC_3': 'G8',
 '108_HBC_mid_1': 'G9',
 '108_HBC_mid_2': 'G10',
 '108_HBC_mid_3': 'G11',
 '108_CCT_1': 'H3',
 '108_CCT_2': 'H4',
 '108_CCT_3': 'H5',
 '108_VEC_1': 'H6',
 '108_VEC_2': 'H7',
 '108_VEC_3': 'H8',
 '108_Str_1': 'H9',
 '108_Str_2': 'H10',
 '108_Str_3': 'H11',
 '187_STB_nVEC_1': 'D3',
 '187_STB_nVEC_2': 'D4',
 '187_STB_nVEC_3': 'D5',
 '187_STB_gen_1': 'D6',
 '187_STB_gen_2': 'D7',
 '187_STB_gen_3': 'D8',
 '187_CTB_1': 'D9',
 '187_CTB_2': 'D10',
 '187_CTB_3': 'D11',
 '187_CCT_1': 'E3',
 '187_CCT_2': 'E4',
 '187_CCT_3': 'E5',
 '187_VEC_1': 'E6',
 '187_VEC_2': 'E7',
 '187_VEC_3': 'E8',
 '187_str_1': 'E9',
 '187_str_2': 'E10',
 '187_str_3': 'E11'}

In [45]:
display(df)

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C,0,0,187_HBC_mid_3,187_HBC_nVEC_1,187_HBC_nVEC_2,187_HBC_mid_2,187_HBC_nTroph_3,187_HBC_nTroph_1,187_HBC_mid_1,187_HBC_nVEC_3,...,0,0,0,0,0,0,0,0,0,0
D,0,0,187_STB_nVEC_1,187_STB_nVEC_2,187_STB_nVEC_3,187_STB_gen_1,187_STB_gen_2,187_STB_gen_3,187_CTB_1,187_CTB_2,...,0,0,0,0,0,0,0,0,0,0
E,0,0,187_CCT_1,187_CCT_2,187_CCT_3,187_VEC_1,187_VEC_2,187_VEC_3,187_str_1,187_str_2,...,0,0,0,0,0,0,0,0,0,0
F,0,0,108_STB_nVEC_1,108_STB_nVEC_2,108_STB_nVEC_3,108_STB_gen_1,108_STB_gen_2,108_STB_gen_3,108_CTB_1_1,108_CTB_2_1,...,0,0,0,0,0,0,0,0,0,0
G,0,0,108_HBC_nTroph_1,108_HBC_nTroph_2,108_HBC_nTroph_3,108_HBC_nVEC_1,108_HBC_nVEC_2,108_HBC_nVEC_3,108_HBC_mid_1,108_HBC_mid_2,...,0,0,0,0,0,0,0,0,0,0
H,0,0,108_CCT_1,108_CCT_2,108_CCT_3,108_VEC_1,108_VEC_2,108_VEC_3,108_Str_1,108_Str_2,...,0,0,0,0,0,0,0,0,0,0
I,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_csv(index=False)